In [332]:
# Probar la conexion a la instacia de SQL SERVER creada.
import pyodbc
import pandas as pd
import numpy as np
import os
import shutil
import subprocess

server = 'MSI'     
database = 'TEST_SOFTEK'  
username = 'Softek_User'      
password = 'Softek1234'     
connection_string = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'

try:
    connection = pyodbc.connect(connection_string)
    print("Connection established successfully.")
    connection.close()
except pyodbc.Error as e:
    print(f"Error: {str(e)}")

Connection established successfully.


In [340]:
#Rutas de los archivos a trabajar
employees_path = r'C:/Users/Jarvis/Documents/Softek_RH/Prueba_Tecnica/AMBIENTE_PRODUCTIVO/ARCHIVOS_EMPLOYEES/Employees_Recortado.csv'
employee_file = pd.read_csv(employees_path)

salarie_path = r'C:/Users/Jarvis/Documents/Softek_RH/Prueba_Tecnica/AMBIENTE_PRODUCTIVO/ARCHIVOS_SALARIES/Salaries_Recortado.csv'
salarie_file = pd.read_csv(salarie_path)

#Checar si hay valores nulos en "EMPLOYEE_ID" y "DEALERSHIP_ID"
try:
    validar_dealership = ['DEALERSHIP_ID']  
    existe_nulos_dealership = employee_file[validar_dealership].isnull().any(axis=1) | employee_file[validar_dealership].eq('').any(axis=1)
    
    rechazados_dealership = employee_file[existe_nulos_dealership].copy()
    rechazados_dealership['Causa_Rechazo'] = 'Sin Identificador de distribuidor'
    
    validar_employee = ['EMPLOYEE_ID']  
    existe_nulos_employee = employee_file[validar_employee].isnull().any(axis=1) | employee_file[validar_employee].eq('').any(axis=1)
    
    rechazados_employee  = employee_file[existe_nulos_employee].copy()
    rechazados_employee['Causa_Rechazo'] = 'Sin Identificador de empleado'

    correctos= employee_file[~existe_nulos_dealership]
    correctos= correctos[~existe_nulos_employee]
    
    correctos_match = correctos.merge(salarie_file, on = 'EMPLOYEE_ID', how = 'inner')
    
    incorrectos_match = correctos[~correctos['EMPLOYEE_ID'].isin(correctos_match['EMPLOYEE_ID'])]
    incorrectos_match = incorrectos_match.copy()
    incorrectos_match['Causa_Rechazo'] = 'Sin Salario'
    
    csv_rechazados = pd.concat([rechazados_dealership, rechazados_employee, incorrectos_match], ignore_index=True)
    
    csv_rechazados_path = r'C:/Users/Jarvis/Documents/Softek_RH/Prueba_Tecnica/AMBIENTE_PRODUCTIVO/ARCHIVOS_RECHAZOS/RECHAZOS.csv'
#CSV con todos los empleados con una causa de rechazo
    csv_rechazados.to_csv(csv_rechazados_path, index=False)
    
    name_column = correctos_match['FIRST_NAME']+''+correctos_match['LAST_NAME']
    correctos_nombre = pd.DataFrame({'Name': name_column})
    
    correctos_match = correctos_match.drop(['FIRST_NAME', 'LAST_NAME'], axis=1)
    df_formato = pd.concat([correctos_match, correctos_nombre], axis=1)
    
    df_formato[['PHONE NUMBER','FAX NUMBER']] = df_formato[['PHONE NUMBER','FAX NUMBER']].astype(str)  
    df_formato['PHONE NUMBER'] = df_formato['PHONE NUMBER'].apply(lambda x: f"({x[:3]}) {x[3:6]}-{x[6:]}")
    df_formato['FAX NUMBER'] = df_formato['FAX NUMBER'].apply(lambda x: f"({x[:3]}) {x[3:6]}-{x[6:]}")
    
    csv_empleados_nuevos = df_formato[df_formato['HIRE_DATE'] == df_formato['DATE_ENTERED']]
    csv_empleados_cambios =  df_formato[df_formato['HIRE_DATE'] < df_formato['DATE_ENTERED']]
    
    layouts =  {
    'EMPLOYEE_ID': 'Employee_Key',
    'Name': 'Name',    
    'PHONE NUMBER': 'Phone_Number', 
    'FAX NUMBER': 'Fax_Number',  
    'EMAIL': 'Email',    
    'ADDRESS': 'Address', 
    'SALARY': 'Salary',  
    'DEALERSHIP_ID': 'Dealership_Manager', 
    'POSITION TYPE': 'Position_Type',    
    'CITY': 'City',  
    'STATE': 'State',      
    'REGION': 'Region',  
    'COUNTRY': 'Country'}
    
    csv_empleados_nuevos = csv_empleados_nuevos.rename(columns=layouts)
    csv_empleados_cambios = csv_empleados_cambios.rename(columns=layouts)
    
    order = [
    'Employee_Key',
    'Name',
    'Phone_Number',
    'Fax_Number',
    'Email',
    'Address',
    'Salary',
    'Dealership_Manager',
    'Position_Type',
    'City',
    'State',
    'Region',
    'Country']
    
    csv_empleados_nuevos = csv_empleados_nuevos [order]
    csv_empleados_cambios = csv_empleados_cambios [order]
    
    csv_empleados_nuevos_path = r'C:/Users/Jarvis/Documents/Softek_RH/Prueba_Tecnica/AMBIENTE_PRODUCTIVO/ARCHIVOS_EMPLOYEES/Employees_nuevos.csv'
#CSV con los empleados nuevos
    csv_empleados_nuevos.to_csv(csv_empleados_nuevos_path, index=False)
    icacls_command = f'icacls "{csv_empleados_nuevos_path}" /grant "*S-1-1-0:(F)"'
    subprocess.run(icacls_command, shell=True, check=True)
    
    csv_empleados_cambios_path = r'C:/Users/Jarvis/Documents/Softek_RH/Prueba_Tecnica/AMBIENTE_PRODUCTIVO/ARCHIVOS_EMPLOYEES/Employees_cambios.csv'
#CSV con los empleados con cambios
    csv_empleados_cambios.to_csv(csv_empleados_cambios_path, index=False)
    icacls_command = f'icacls "{csv_empleados_cambios_path}" /grant "*S-1-1-0:(F)"'
    subprocess.run(icacls_command, shell=True, check=True)

    conn = pyodbc.connect(connection_string)
    cursor = conn.cursor()
    
#Carga de CSV's
    bulk_insert_query = f"""
    BULK INSERT TEST.HISTORY_DIM_EMPLOYEE
    FROM '{csv_empleados_nuevos_path}'
    WITH (
        FIRSTROW = 2,           -- Skip the header row if present
        FIELDTERMINATOR = ',',  -- Field delimiter in your CSV
        ROWTERMINATOR = '\\n'    -- Row terminator in your CSV
    );
    """
    cursor.execute(bulk_insert_query)
    conn.commit()
    
    bulk_insert_query = f"""
    BULK INSERT TEST.DIM_EMPLOYEE
    FROM '{csv_empleados_nuevos_path}'
    WITH (
        FIRSTROW = 2,           -- Skip the header row if present
        FIELDTERMINATOR = ',',  -- Field delimiter in your CSV
        ROWTERMINATOR = '\\n'    -- Row terminator in your CSV
    );
    """
    cursor.execute(bulk_insert_query)
    conn.commit()
    
    bulk_insert_query = f"""
    BULK INSERT TEST.HISTORY_DIM_EMPLOYEE
    FROM '{csv_empleados_cambios_path}'
    WITH (
        FIRSTROW = 2,           -- Skip the header row if present
        FIELDTERMINATOR = ',',  -- Field delimiter in your CSV
        ROWTERMINATOR = '\\n'    -- Row terminator in your CSV
    );
    USE TEST_SOFTEK;
    TRUNCATE TABLE [TEST].[DIM_EMPLOYEE_CAMBIOS];
    BULK INSERT TEST.DIM_EMPLOYEE_CAMBIOS
    FROM '{csv_empleados_cambios_path}'
    WITH (
        FIRSTROW = 2,           -- Skip the header row if present
        FIELDTERMINATOR = ',',  -- Field delimiter in your CSV
        ROWTERMINATOR = '\\n'    -- Row terminator in your CSV
    );
    MERGE INTO TEST.DIM_EMPLOYEE AS target
    USING TEST.DIM_EMPLOYEE_CAMBIOS AS source
    ON target.Employee_Key = source.Employee_Key
    WHEN MATCHED THEN
        DELETE;
        INSERT INTO TEST.DIM_EMPLOYEE
        SELECT * FROM TEST.DIM_EMPLOYEE_CAMBIOS;
    """
    cursor.execute(bulk_insert_query)
    conn.commit()
    
    cursor.close()
    conn.close()
    
    origen_employees = "C:/Users/Jarvis/Documents/Softek_RH/Prueba_Tecnica/AMBIENTE_PRODUCTIVO/ARCHIVOS_EMPLOYEES"
    destino= "C:/Users/Jarvis/Documents/Softek_RH/Prueba_Tecnica/AMBIENTE_PRODUCTIVO/ARCHIVOS_CARGADOS"
    nombre_archivos = "Employees"
    archivos_employee = os.listdir(origen_employees)
    
#Mover los archivos a una carpeta de procesados   
    for archivo in archivos_employee:
        if archivo.startswith(nombre_archivos):
            origen_completo = os.path.join(origen_employees, archivo)
            destino_completo = os.path.join(destino, archivo)
            shutil.move(origen_completo, destino_completo)
            
    origen_salaries = "C:/Users/Jarvis/Documents/Softek_RH/Prueba_Tecnica/AMBIENTE_PRODUCTIVO/ARCHIVOS_SALARIES"
    nombre_salaries = "Salaries"
    archivos_salaries = os.listdir(origen_salaries)
    
    for archivo in archivos_salaries:
        if archivo.startswith(nombre_salaries):
            origen_completo = os.path.join(origen_salaries, archivo)
            destino_completo = os.path.join(destino, archivo)
            shutil.move(origen_completo, destino_completo)    
    print('ETL Finalizo exitosamente')    
except Exception as e:
    print(f"An error occurred: {str(e)}")


ETL Finalizo exitosamente


C:\Users\Jarvis\AppData\Local\Temp/ipykernel_31512/2478313769.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  correctos= correctos[~existe_nulos_employee]
